In [1]:
# pip install accelerate
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import requests
import torch

model_id = "google/medgemma-27b-it"

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful medical assistant."}]
    },
    {
        "role": "user",
        "content": [{"type": "text", "text": "How do you differentiate bacterial from viral pneumonia?"}]
    }
]

inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=200, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)


/home/tp030/miniforge3/envs/medgemma/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 12/12 [00:15<00:00,  1.29s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Okay, as a helpful medical assistant, I can explain the key differences between bacterial and viral pneumonia. It's important to remember that a definitive diagnosis often requires specific tests, but clinical presentation and certain risk factors can help guide the initial assessment.

Here's a breakdown of how we typically differentiate between the two:

**1. Causative Agent:**

*   **Bacterial Pneumonia:** Caused by bacteria (e.g., *Streptococcus pneumoniae*, *Haemophilus influenzae*, *Mycoplasma pneumoniae*, *Staphylococcus aureus*, *Klebsiella pneumoniae*).
*   **Viral Pneumonia:** Caused by viruses (e.g., Influenza virus, Respiratory Syncytial Virus (RSV), Adenovirus, Rhinovirus, Coronavirus (including SARS-CoV-2)).

**2. Onset and Progression:**

*   **Bacterial Pneumonia:** Often has a more **abrupt** onset. Symptoms tend to develop relatively quickly (


In [2]:

# Image attribution: Stillwaterising, CC0, via Wikimedia Commons
image_url = "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are an expert radiologist."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Describe this X-ray"},
            {"type": "image", "image": image}
        ]
    }
]

inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=200, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)


Okay, let's analyze this chest X-ray.

**Overall Impression:**

This is a standard posteroanterior (PA) view of the chest. The image quality appears good, with adequate penetration and contrast. The patient is positioned well, allowing for a clear view of the lungs, heart, mediastinum, and bony structures.

**Detailed Breakdown:**

1.  **Lungs:**
    *   **Parenchyma:** The lung fields appear clear bilaterally. There are no obvious signs of consolidation (pneumonia), masses, nodules, or significant interstitial changes. The vascular markings are normal in caliber and distribution.
    *   **Pleura:** No pleural effusions (fluid collections) or pneumothorax (collapsed lung) are visible. The costophrenic angles (where the diaphragm meets the ribs) are sharp and clear.
    *   **Hila:** The hila (where the major bronchi and vessels enter the lungs) appear normal in
